In [ ]:
import math
import random
from itertools import chain

global k_fold

sc = SparkContext.getOrCreate()

TRAIN_TEST = 0.8
k_fold=5

global n_columns
global w
global b
global n_row
global y_h_m_y


n_columns=56 #total number of columns (58) - the one deleted (57) - label (56)
col_sums=[]
averages=[]
sigmas=[]
w=[]
b=0



def initializeAccumulators():
    i=0
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def addToAccumulators(row):
    if(len(row)!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
    i=0
    while(i<n_columns):
        print("Len row: "+str(len(row))+" n_accomulators"+str(len(col_sums)))
        col_sums[i].add(row[i])
        i+=1
        
def preprocessing(row):
    #splitting
    split=row.split(" ")
    label = int(split[57])
    del split[57] #label
    del split[56] #col 57th
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    #return
    return (key,train,split,label)
def calcAvg(n_row):
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1
def calcResiduals(row):
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row
def calcSigmas(n_row):
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1
def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row

def initializeWeights(random_init=False):
    if(random_init):
        #return sc.parallelize([(i, random.random()) for i in range(0,n_columns)])
        return [random.random() for i in range(0,n_columns)]
    else:
        #return sc.parallelize([(i, 0.0) for i in range(0,n_columns)])
        return [0.0 for i in range(0,n_columns)]

def initializeBias():
    return sc.parallelize(0.0)

def sigmoid(z):
    return 1/(1+math.exp(-z))

def predict(w,b,X):
    return sigmoid(sum([X[i]*w[i] for i in range(len(w))])+b)

def predict_parallel(w,b,X):
    #Change X representation
    X = sc.parallelize(X).zipWithIndex().map(lambda x: (x[1],x[0]))
    wX_plus_b=X.join(w).map(lambda x: (x[1][0]*x[1][1])+b).sum()
    return sigmoid(wX_plus_b)

def compute_cost(dataset,w,b,lambda_reg):
    return (-1/dataset.count())*dataset.map(lambda x: x[3]*math.log(predict(w,b,x[2])) \
                                    +(1-x[3])*math.log(1-predict(w,b,x[2]))).sum() \
                                    + lambda_reg/(2*dataset.count())*sum([i*i for i in w]);

def make_folds(x):
    return (x[0],int(random.random()*10-1),x[1],x[2])
     
    
def train(filename, iterations, learning_rate, lambda_reg):
    global w
    global b
    initializeAccumulators()
    dataset=sc.textFile(filename).map(preprocessing).sortBy(lambda x: x[0])
    n_row=dataset.count()
#    print(dataset.first())
    calcAvg(n_row)
    dataset.map(calcResiduals).collect()
    calcSigmas(n_row)
    dataset=dataset.map(normalize)
#    print(dataset.first())
    train=dataset.filter(lambda x: x[1]==1)
    train_size=train.count()
#    train_x=train.map(lambda x: x[2])
#    train_y=train.map(lambda x: x[3])
    test=dataset.filter(lambda x: x[1]==0)
#    test_x=test.map(lambda x: x[2])
#    test_y=test.map(lambda x: x[3])
    print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")
    fold_length=train.count()/k_fold
    #Folds making
    costs_fold=[]
    for i_fold in range(k_fold):
        starting_fold=fold_length*i_fold
        end_fold=starting_fold+fold_length
        test_fold=train.zipWithIndex().filter(lambda t: (t[1]>=starting_fold and t[1]<end_fold)).map(lambda t: t[0]) #the map get rid of the index again
        
        train_fold=train.zipWithIndex().filter(lambda t: t[1]<starting_fold or t[1]>=end_fold).map(lambda t: t[0])
        train_fold_size=train_fold.count()
        #print("Test fold:")
        #print(test_fold.take(1))
        #print("Train fold:")
        #print(train_fold.take(1))
        w=initializeWeights()
        dw=[i for i in range(0,n_columns)]
        #Gradient descent
        costs=[]
        for iteration in range(iterations):
            for j in range(n_columns):
                X_j=train_fold.map(lambda x: (predict(w,b,x[2])-x[3])*x[2][j]).sum()
                dw[j]=(1/train_fold_size)*X_j+(lambda_reg/train_fold_size)+w[j]
                w[j]-=learning_rate*dw[j]
            b-=learning_rate*(1/train_fold_size)*train_fold.map(lambda x: predict(w,b,x[2])-x[3]).sum()
            cost=compute_cost(train_fold,w,b,lambda_reg)
            costs.append(cost)
            print("-> Iteration done: "+str(iteration+1)+" of "+str(iterations)+". Cost: "+str(cost))
        costs_fold.append(costs)
        print("--> Fold #"+str(i_fold+1)+" of "+str(k_fold)+" is done. Average cost: "+str(sum(costs)/iterations))
    
    costs_fold=list(chain.from_iterable(costs_fold))
    print("---> "+str(k_fold)+"-fold validation done. Average cost: "+str(sum(costs_fold)/(iterations*k_fold)))
            
    
    
train("dataset/spam.data",5,0.1,0)

Split train/test done. Train contains 3717 elements, Test contains 884 elements
-> Iteration done: 1 of 5. Cost: 0.6429051336857715
-> Iteration done: 2 of 5. Cost: 0.6116104380482642


In [22]:
def predict(w,b,X):
    #Change X representation
    X = X.zipWithIndex().map(lambda x: (x[1],x[0]))
    wX_plus_b=X.join(w).map(lambda x: (x[1][0]*x[1][1])+b).sum()
    return sigmoid(wX_plus_b)
predict(initializeWeights(), 0, sc.parallelize([i for i in range(n_columns)]))

0.5

In [84]:
def gradientDecent(w,iterations,learning_rate,lambda_reg):
    dW=sc.parallelize([0 for i in range(n_columns)])
    
        
    

SyntaxError: invalid syntax (<ipython-input-84-a4c43414cb95>, line 2)

In [13]:
#t=zip([] for i in range(0,n_columns),"w"+str(i) for i in range(0,n_columns))
t=set(zip([i for i in range(0,n_columns)],[i for i in range(0,n_columns)]))
t

NameError: name 'filename' is not defined

In [ ]:
def transpose(x,y):
    for i in range(0,k):
        x[0][i]+=y[i]
train.aggregate(([[] for i in range(0,n_columns)]],[]), transpose, globalize)




In [5]:
w

[0.02969887985785198,
 -0.008284080268595848,
 0.041822455051646536,
 0.017256782364696212,
 0.05326246112325493,
 0.05525192221253085,
 0.08359049881679019,
 0.05357137573372938,
 0.05334075512061355,
 0.032029808065755165,
 0.05258743991967035,
 0.0074777587120948945,
 0.030843601386010843,
 0.015026309504026104,
 0.04391438471523303,
 0.06214018080185739,
 0.058092914082291607,
 0.04612961025274813,
 0.037837685771370554,
 0.04307990244029978,
 0.06509162065659488,
 0.027069961330382444,
 0.07540793938931052,
 0.051334684864959006,
 -0.04792783878428576,
 -0.04535069147091169,
 -0.04060001080916476,
 -0.02602467970098737,
 -0.02396264602748511,
 -0.030323902587659126,
 -0.020458479099227208,
 -0.017709351797302927,
 -0.029095141822853526,
 -0.01674092615162573,
 -0.02503572963510947,
 -0.02085794067984527,
 -0.033570272590953215,
 -0.009213712049893038,
 -0.025080441292352786,
 -0.005469604052854424,
 -0.02086594317751748,
 -0.029687397143216786,
 -0.028632641677854224,
 -0.02155505